In [1]:
# 导入必要的包

In [2]:
import torch # pytorch 主库
from torch import nn # 神经网络
from torch.nn import functional as F # 激活函数
from d2l import torch as d2l # 本书自定义的库
import random # 随机数
import pandas as pd # 读取数据
import numpy as np # 数组
import math

In [3]:
# 采样器
class SeqDataLoader:
    """加载序列数据的迭代器"""
    # 输入 batch_size, num_steps, use_random_iter, max_tokens，返回按顺序或者随机的方式小样本抽样的一个迭代器
    def __init__(self, batch_size, num_steps, use_random_iter, max_tokens):
        # max_tokens是指读取 time_series 的最大数量，截取前max_tokens个，目前弃用状态
        if use_random_iter:
            self.data_iter_fn = seq_data_iter_random
        # 如果use_random_iter为True，则使用随机抽样
        else:
            self.data_iter_fn = seq_data_iter_sequential
        # 否则使用顺序抽样
        self.time_series = load_data_timeseries(max_tokens)
        # 读取时光机器数据集
        # 输入的max_tokens是指读取 time_series 的最大数量
        self.batch_size, self.num_steps = batch_size, num_steps

    def __iter__(self):
        return self.data_iter_fn(self.time_series, self.batch_size, self.num_steps)
    # 返回迭代器

def load_data_timeseries(max_tokens=-1):
    """ 返回时间序列数据集 """
    time_series = pd.read_csv('/home/ubuntu/code/EAST/data/2024-09-20/POINT_N/shot_77003_data.csv')

    if max_tokens > 0:
        time_series = time_series[:max_tokens]
    return time_series.iloc[:,1:].values
    # 如果 max_tokens 大于 0，则返回前 max_tokens 个词元索引，否则返回所有索引


# 顺序抽样
def seq_data_iter_sequential(time_series, batch_size, num_steps):
    """使用顺序分区生成一个小批量子序列"""
    """ 输入语料库，批量大小，时间步数，返回小批量子序列"""
    
    offset = random.randint(0, num_steps)
    # 从随机偏移量开始划分序列
    num_tokens = ((len(time_series) - offset - 1) // batch_size) * batch_size
    # 计算有多少个有效的样本
    # 为什么要这么计算呢？
    ## 首先是搞了个偏移量，让我们的结果有随机性，生成的小批量子序列不同
    ## 其次我们最终选到的数据实际上是 batch_size 的整数倍，因此 num_tokens 是有效样本量
    Xs = torch.tensor(time_series[offset: offset + num_tokens, :])
    Ys = torch.tensor(time_series[offset + 1: offset + 1 + num_tokens, :])
    # 计算有效的输入和标签
    # 我们的 time_series 是 (样本量, 特征) 的格式
    Xs, Ys = Xs.reshape(batch_size, -1, 11), Ys.reshape(batch_size, -1, 11)
    # 将输入和标签转换为 (小样本量, -1, 特征数) 的格式
    num_batches = Xs.shape[1] // num_steps
    # 计算有多少个批量
    for i in range(0, num_steps * num_batches, num_steps):
        X = Xs[:, i: i + num_steps, :]
        Y = Ys[:, i: i + num_steps, :]
        yield X, Y

def seq_data_iter_random(time_series, batch_size, num_steps):
    """使用随机抽样生成一个小批量子序列"""
    
    time_series = time_series[random.randint(0, num_steps - 1):]
    # 从随机偏移量开始对序列进行分区，随机范围为[0, num_steps - 1]
    # 减去1，是因为我们需要考虑标签
    # random.randint(a, b) 返回一个随机整数N，a <= N <= b

    num_subseqs = (len(time_series) - 1) // num_steps
    # 一个子序列的长度为num_steps，所以有len(corpus) - 1 // num_steps 个子序列
    # // 的意思是整除
    # 减去1，是因为我们需要考虑标签

    initial_indices = list(range(0, num_subseqs * num_steps, num_steps))
    # 生成一个包含所有子序列的起始索引的列表
    # range(start, end, step) 返回一个包含等差数列的列表

    random.shuffle(initial_indices)
    # 将所有子序列的起始索引打乱
    # random.shuffle() 方法将序列的所有元素随机排序
    # 在随机抽样的迭代过程中，
    # 来自两个相邻的、随机的、小批量中的子序列不一定在原始序列上相邻

    def data(pos):
        return time_series[pos: pos + num_steps, :]
        # 返回从pos位置开始的长度为num_steps的序列

    num_batches = num_subseqs // batch_size
    # 一个批量的子序列的数量为num_subseqs // batch_size

    for i in range(0, batch_size * num_batches, batch_size):
        
        initial_indices_per_batch = initial_indices[i: i + batch_size]
        # 在这里，initial_indices包含子序列的随机起始索引
        # initial_indices_per_batch包含了一个批量的子序列的随机起始索引
        
        X = [data(j) for j in initial_indices_per_batch]
        Y = [data(j + 1) for j in initial_indices_per_batch]
        yield torch.tensor(X), torch.tensor(Y)
        # X, Y 的格式是 (batch_size, num_steps, 特征数)


In [ ]:
# 读取数据
# 至于我们要加载哪些数据，已经内置在 SeqDataLoader 类中，更进一步load_data_timeseries 函数中
def load_data_east(batch_size, num_steps, 
                           use_random_iter=False, max_tokens=10000):
    # use_random_iter是指是否使用随机抽样
    # max_tokens是指将corpus中的词转换为标记的最大数量，截取前max_tokens个词
    
    """返回时光机器数据集的迭代器和词表"""
    data_iter = SeqDataLoader(
        batch_size, num_steps, use_random_iter, max_tokens)
    # 创建一个SeqDataLoader对象
    # 输入 batch_size, num_steps, use_random_iter, max_tokens，返回按顺序或者随机的方式小样本抽样的一个迭代器
    return data_iter